In [215]:
using LowLevelFEM

gmsh.initialize();

In [216]:
gmsh.open("plate_with_hole_tria_1st_size-def.geo")
#gmsh.open("plate_with_hole_tria_1st_size-5.geo")
#gmsh.open("plate_with_hole_tria_1st_size-1.geo")

#gmsh.open("plate_with_hole_tria_1st_size-graded.geo")

#gmsh.open("plate_with_hole_tria_2nd_size-def.geo")
#gmsh.open("plate_with_hole_tria_3rd_size-def.geo")
#gmsh.open("plate_with_hole_tria_5th_size-def.geo")

#gmsh.open("plate_with_hole_quad_1st_size-def.geo")

#gmsh.open("plate_with_hole_quad_1st_size-5.geo")
#gmsh.open("plate_with_hole_quad_2nd_size-5.geo")

#gmsh.open("plate_with_hole_quad_5st_size-1-graded.geo")

#gmsh.open("plate_with_hole_quad_1st_size-regular.geo")
#gmsh.open("plate_with_hole_quad_2nd_size-regular.geo")
#gmsh.open("plate_with_hole_quad_5th_size-regular.geo")

#gmsh.open("plate_with_hole_quad_1st_size-regular.geo")
#gmsh.open("plate_with_hole_quad_2nd_size-regular.geo")
#gmsh.open("plate_with_hole_quad_3rd_size-regular.geo")
#gmsh.open("plate_with_hole_quad_5th_size-regular.geo")

#openPreProcessor()

mat = material("body")
problem = Problem([mat], type=:PlaneStress)

Info    : Reading 'plate_with_hole_tria_1st_size-def.geo'...
Info    : Meshing 1D...nts                                                                                                          
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Ellipse)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.00100502s, CPU 0.001005s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00382628s, CPU 0.003824s)
Info    : 94 nodes 201 elements
Info    : Done reading 'plate_with_hole_tria_1st_size-def.geo'


Problem("plate_with_hole_tria_1st_size-def", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 94, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [217]:
supp1 = displacementConstraint("bottom", uy=0)
supp2 = displacementConstraint("left", ux=0)
load1 = load("top", fy=1);

In [218]:
q = solveDisplacement(problem, load=[load1], support=[supp1, supp2])
S = solveStress(q);

In [219]:
#u = showDoFResults(q, name="uvec")
#ux = showDoFResults(q, :ux, name="ux")
#uy = showDoFResults(q, :uy, name="uy")

s1 = showStressResults(S, name="σ", visible=true, smooth=false)
s2 = showStressResults(S, name="σ smooth", smooth=true)
#sx = showStressResults(S, :sx, name="σx", smooth=true)
#sy = showStressResults(S, :sy, name="σy", smooth=true)
#synsm = showStressResults(S, :sy, name="σy non-smooth", smooth=false)
#sxy = showStressResults(S, :sxy, name="τxy", smooth=true)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


1

In [220]:
gmsh.view.option.setNumber(s1, "NormalRaise", 10)
gmsh.view.option.setNumber(s2, "NormalRaise", 10)

gmsh.view.option.setNumber(s1, "ShowElement", 1)
gmsh.view.option.setNumber(s2, "ShowElement", 1)

In [221]:
err = fieldError(S)
serr = showDoFResults(err, name="szakadási hiba")

2

In [222]:
f3D = -S ⋅ ∇
f2D = projectTo2D(f3D)
showDoFResults(f2D, name="belső erők hibája")

3

In [223]:
n1y = VectorField(problem, "top", [0, 1, 0])
S1 = elementsToNodes(S)
S2 = nodesToElements(S1, onPhysicalGroup="top")
p3D = S2 * n1y

n1x = VectorField(problem, "top", [1, 0, 0])
p1x = p3D * n1x
p1y = p3D * n1y
f1f = load("top", fx=p1x, fy=p1y)
f2D = loadVector(problem, [f1f])
f = loadVector(problem, [load1])

showDoFResults(f, name="eredeti terhelés")
showDoFResults(f2D, name="terhelés feszültségből")
showDoFResults(f - f2D, name="din. peremfelt. hiba")

6

In [224]:
vx(x, y, z) = -x / √(x^2 + y^2)
vy(x, y, z) = -y / √(x^2 + y^2)
n2 = VectorField(problem, "hole", [vx, vy, 1])
S3 = nodesToElements(S1, onPhysicalGroup="hole")
p2 = S3 * n2
showElementResults(projectTo2D(p2), name="din. peremfelt. hiba - furat")

7

In [225]:
ex = VectorField(problem, "body", [1, 0, 0])
ey = VectorField(problem, "body", [0, 1, 0])

fx = f3D ⋅ ex
fy = f3D ⋅ ey

load2 = load("body", fx=fx, fy=fy)

q3 = solveDisplacement(problem, load=[load1, load2], support=[supp1, supp2])
S3 = solveStress(q3)

s3 = showStressResults(S3, name="javított fesz.")

gmsh.view.option.setNumber(s3, "NormalRaise", 10)
gmsh.view.option.setNumber(s3, "ShowElement", 1)

In [226]:
f3 = -S3 ⋅ ∇
f3_2D = projectTo2D(f3)
showDoFResults(f3_2D, name="belső erők hibája 2")

9

In [227]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [228]:
gmsh.finalize()

In [229]:
N = 1
σr = -N / 2 * (3 - 4 + 1)
σφ = N / 2 * (1 + 1) + N / 2 * (3 + 1)

3.0